<a href="https://colab.research.google.com/github/K-Viera/Python/blob/main/IA/GeneticAlgorithms/Taller_Algor%C3%ADtmos_Gen%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Taller Algorítmos Genéticos
-Simón Echeverri \
-Camilo Zapata \
-Kevin Viera

In [125]:
import numpy as np
import pandas as pd
import plotly.express as px
import random
import matplotlib.pyplot as plt

##Punto 1

f(x) (de la que se habla en el enunciado del problema) es de la forma 
f(x) = a1x1 + a2x2 + ... + am-1xm-1 - b

Buscamos estimar los parámetros ai y b. La serie de parámetros (un arreglo con ellos) será entonces nuestro cromosoma, así:

\

cromosoma = [a1, a2, a3, ... , am-1, b]

\

Nuestros cromosomas serán generados aleatoriamente, aunque se podrían hacer acotaciones en los valores de las as y de la b para obtener mejores de una vez (por ejemplo que tengan un rango de generación específico).

\

Contamos con una serie de arreglos (N en total) que tienen M componentes binarios que describen un objeto de un dominio, donde la M-ésima componente (última componente) corresponde a la clase de un objeto, que puede valer 1, 2 o 3.

\

La idea es que f(x) estime la clase del objeto dados los componentes binarios. Estos componentes binarios corresponden a los xi de la función.

\

Una función ejemplo entonces sería:

\

Dados:

a1 = 3, a2 = 1, a3 = 1 y b = 2 (cromosoma)\
x1 = 1, x2 = 0, x3 = 0

f(x) = 3x1 + 1x0 + 1x0 - 2 = 1

\

Según esta función de ejemplo, un vector con las componentes binarias dadas (las xs), corresponden a un objeto de clase 1.

\

Como buscamos que f(x) sea capaz de identificar la clase de un objeto cualquiera dados sus componentes binarios, la función será valorada (funcion fitness) a partir de cuántos resultados correctos tenga al ser evaluada con los N arreglos que YA poseen la clase en su M-ésima posición.

\

Así, si f(x) evaluada en los N arreglos tiene 4 resultados correctos (es decir, acertó 4 clases en total) va a tener una valoración de 4.

La función que más resultados correctos tenga, será la mejor prediciendo la clase de un objeto dados solos sus componentes binarios.

\

Los cromosomas pueden mutar al variar un valor en cualquier índice, utilizando el método de punto de corte. Es decir que puede cambiar ai o b, y eso generaría un nuevo individuo.

\

El cruce también se puede generar con el método de punto de corte. Donde se tomarían las primeras ai de un padre y se unirían con las ai restantes del otro padre.



##Punto 2


###Cromosomas


A) Cada uno de nuestros cromosomas van a ser un arreglo de 6 posiciones (deberian ser 8 pero como la primera y ultima posicion son estáticas solo van a variar 6 de ellas) en la que cada una de las posiciones del arreglo nos indican que letra va dentro de ella de la siguiente manera

In [126]:
# las letras van en orden
valLetras=[('A',10),('B',3),('C',7),('D',1),('E',2),('F',8),('G',4),('H',10),('I',9),('J',3)]

###Generacion poblacional


se crean dos formas de generacion poblacional, una en la que se pueden repetir las letras y otra en la que las letras no pueden ser repetidas

la primera seleccion repetible simplemente genera un arreglo de N posiciones (para efectos del ejercicio N=6) y a cada una de estas posiciones le asigna aleatoriamente una de las psiciones de arreglo de la lista de letras

In [127]:
#Poblacion que permite repetir letras
def generateFirstPopulation(N, NP):
  population =[np.random.randint(0, len(valLetras), N) for _ in range(NP)]
  return population

la segunda seleccion sin repetibles genera un estado inicial por cada individuo de la poblacion con la lista de letras disponible y las va seleccionando y eliminado de este a medida que va agregando valores en la poblacion

In [128]:
#Poblacion que no permite repetir numeros
def generateFirstPopulationSR(NP):
  population=[]
  for _ in range(NP):
    disponible=[1,2,3,4,5,6,7,8]
    individual=[]
    while len(individual)!=6:
      rand=random.randint(0,len(disponible)-1)
      letra=disponible[rand]
      individual.append(letra)
      disponible.remove(letra)
    population.append(individual)
  return population

###Funcion de evaluacion


la funcion de evaluacion primero multiplica los adyacentes (en caso de ser la primera y ultima posicion del arreglo lo multiplicamos por el valor estatico que nos indicaba el enunciado la primera con A y el ultimo con J) para luego sumarlos a la cantidad o suma total

In [129]:
def fitnessFunct(x):
  sum=0
  inital=10*valLetras[x[0]][1]
  final=3*valLetras[x[len(x)-1]][1]
  sum+=inital+final
  for i,letra in enumerate(x):
    actualSum=valLetras[letra][1]
    if i>0:
      actualSum*=valLetras[x[i-1]][1]
    else:
      #significa que el anterior siempre es b
      actualSum*=10
    if i<len(x)-1:
      actualSum*=valLetras[x[i+1]][1]
    else:
      #significa que el siguiente siempre es j
      actualSum*=3
    sum+=actualSum
  return sum

In [130]:
def roulette(population):
  of=[fitnessFunct(s) for s in population]
  suma=np.int64(sum(of))
  probability=of/suma
  # print(probability)
  accumulatedProb=np.cumsum(probability)
  return accumulatedProb

In [131]:
def selection(population):
  nr=np.random.uniform()
  for i,p in enumerate(roulette(population)):
    if nr<p:
      return i, population[i]

In [132]:
def crossover(ind1, ind2):
  cp=np.random.randint(1,len(ind1)-1)
  ## Create new children
  child1=np.concatenate((ind1[0:cp+1],ind2[cp+1:len(ind2)]))
  child2=np.concatenate((ind2[0:cp+1],ind1[cp+1:len(ind1)]))
  return child1,child2

In [133]:
def mutate(ind):
  cp=np.random.randint(0,len(ind)-1)
  child=np.copy(ind)
  child[cp]=np.random.randint(0,len(valLetras))
  return child

In [156]:
def geneticAlgorithm(sizeIndividual, sizePopulation, generations, mutation_probability, crossover_probability):
  #Genera la poblacion con una cantidad de cromosomas de individual y una poblacion de population
  population=generateFirstPopulation(sizeIndividual, sizePopulation)
  #Genera la probabilidad acumulada (sin ordenar) para toda la poblacion
  accumulatedProb=roulette(population)
  print("first population:",population)
  print("accumulated Prob:",accumulatedProb)
  of=[fitnessFunct(s) for s in population]
  sorted_population=sorted(population,key=fitnessFunct,reverse=True)
  bestIndividual=[sorted_population[0]] # Lista de los mejores cromosomas
  for i in range(generations):
    #Generar la nueva población
    nextPopulation=[]
    count=0
    while True: # para crear la población cuando tengamos NP individuos
      # Hacemos selección de dos individuos y cruce si es posible
      ind1=selection(population)[1]
      ind2=selection(population)[1]
      crossProb=nr=np.random.uniform()
      if crossProb<crossover_probability:
        child1,child2=crossover(ind1,ind2)
        print(ind1,ind2)
        nextPopulation.append(child1)
        nextPopulation.append(child2)
        print('cross',nextPopulation)
      else:
        nextPopulation.append(ind1)
        nextPopulation.append(ind2)
      print('1',nextPopulation)
      # Hacemos selección de un individuo y mutación si es posible
      mutProb=nr=np.random.uniform()
      ind=selection(population)[1]
      if mutProb<mutation_probability:
        child=mutate(ind)
        nextPopulation.append(child)
      else:
        nextPopulation.append(ind)
      print('2',nextPopulation)
      if len(nextPopulation)>=sizePopulation:
        nextPopulation=nextPopulation[0:sizePopulation]
        break;
      print('3',nextPopulation)
    
    accumulatedProb=roulette(nextPopulation)
    sorted_population=sorted(nextPopulation,key=fitnessFunct,reverse=True)
    print('poblacion generacion', i,nextPopulation)
    print('___________________________________________________________________')
    print()

    bestIndividual.append(sorted_population[0]) 
    population=nextPopulation
  sortedBest=sorted(bestIndividual,key=fitnessFunct,reverse=True) 
  return bestIndividual,sortedBest[0]

###Pruebas


In [164]:
print(fitnessFunct([0, 5, 5, 9, 7, 8]))
print(fitnessFunct([7, 2, 8, 9, 0, 7]))
print(fitnessFunct([0, 7, 0, 0, 8, 7]))
print(fitnessFunct([0, 0, 0, 0, 0, 0]))

2539
2519
5200
5430


In [162]:
bestList,bestInd=geneticAlgorithm(6, 10, 30, 0.5, 0.5)
bestInd

first population: [array([2, 4, 7, 9, 8, 7]), array([2, 2, 1, 4, 9, 6]), array([0, 7, 6, 3, 1, 1]), array([9, 2, 4, 0, 4, 7]), array([4, 5, 0, 2, 3, 2]), array([3, 3, 9, 4, 4, 5]), array([6, 1, 9, 3, 4, 5]), array([3, 8, 1, 8, 8, 6]), array([9, 5, 0, 4, 0, 1]), array([7, 7, 3, 3, 4, 3])]
accumulated Prob: [0.13510592 0.22578902 0.39840035 0.47968007 0.59435798 0.61003026
 0.6423476  0.75659317 0.86781236 1.        ]
1 [array([2, 4, 7, 9, 8, 7]), array([2, 4, 7, 9, 8, 7])]
2 [array([2, 4, 7, 9, 8, 7]), array([2, 4, 7, 9, 8, 7]), array([7, 7, 3, 3, 4, 3])]
3 [array([2, 4, 7, 9, 8, 7]), array([2, 4, 7, 9, 8, 7]), array([7, 7, 3, 3, 4, 3])]
1 [array([2, 4, 7, 9, 8, 7]), array([2, 4, 7, 9, 8, 7]), array([7, 7, 3, 3, 4, 3]), array([0, 7, 6, 3, 1, 1]), array([9, 5, 0, 4, 0, 1])]
2 [array([2, 4, 7, 9, 8, 7]), array([2, 4, 7, 9, 8, 7]), array([7, 7, 3, 3, 4, 3]), array([0, 7, 6, 3, 1, 1]), array([9, 5, 0, 4, 0, 1]), array([4, 7, 6, 3, 1, 1])]
3 [array([2, 4, 7, 9, 8, 7]), array([2, 4, 7, 9, 8, 

array([0, 7, 0, 0, 8, 7])

In [137]:
population=generateFirstPopulation(6,5)
print(population)
# print(pop[0])
# print(fitnessFunct(pop[0]))
print(roulette(population))
selection(population)[1]


[array([5, 6, 8, 9, 8, 6]), array([3, 9, 9, 4, 7, 0]), array([0, 5, 7, 4, 2, 4]), array([2, 4, 3, 9, 8, 7]), array([3, 5, 0, 8, 2, 2])]
[0.18214491 0.27659574 0.57504313 0.69393329 1.        ]


array([0, 5, 7, 4, 2, 4])

In [138]:
ind1=selection(population)[1]
ind2=selection(population)[1]
print(ind1)
print(ind2)
crossover(ind1,ind2)

[0 5 7 4 2 4]
[0 5 7 4 2 4]


(array([0, 5, 7, 4, 2, 4]), array([0, 5, 7, 4, 2, 4]))

In [139]:
popSR=generateFirstPopulationSR(5)
print(popSR)
print(popSR[0])
print(fitnessFunct(popSR[0]))

[[2, 5, 6, 8, 3, 4], [4, 8, 2, 6, 7, 3], [1, 7, 6, 5, 4, 3], [7, 1, 8, 6, 2, 4], [5, 4, 8, 1, 2, 7]]
[2, 5, 6, 8, 3, 4]
1208


##Punto 3

###Definiciones

Cantidad de máquinas -> M \
Cantidad de piezas -> N \
Matriz de tiempos NxM \
Índice de pieza -> i \
Índice de máquina -> j 

###Cromosoma

Nuestro cromosoma será el arreglo de piezas. El índice del arreglo indica qué pieza es, el valor en 
ese índice, indica qué máquina producirá esa pieza. En el ejemplo de abajo, la pieza #1 será producida
en la máquina #4, por lo cual tendrá un tiempo asociado de elaboración de 105.

La matriz de tiempos en la fila i está asociada a la pieza i. Cada valor dentro de esta fila
corresponde al tiempo que le toma ser producida en la máquina j (el índice de columnas es j).

In [140]:
import random


piezas = [3, 4, 2, 4, 5, 0, 1, 2, 5]
tiempos = [[30, 40, 10, 60, 100, 50],
           [35, 45, 15, 65, 105, 55],
           [30, 40, 10, 60, 100, 50],
           [30, 40, 10, 60, 100, 50]]



###Función Fitness

Dado un cromosoma, retorna el tiempo total de producción, haciendo la búsqueda en la matriz de tiempos

In [141]:
def fitness (piezas):
  totalTime = 0
  for i, pieza in enumerate(piezas):
      totalTime = tiempos[i][pieza] + totalTime
  return totalTime

###Método de cruce

Usaremos el método de punto de corte, donde dados 2 cromosomas, para la
generación del primer nuevo individuo, se toman los valores de 0 a cp del primer padre y de cp+1 a N del segundo padre. Para el segundo individuo se hace el proceso inverso. El cp es generado aleatoriamente.

###Mutación

Para mutar también generaremos un punto de corte (cp). Esta vez el punto de corte indicará el índice de la pieza a ser mutada. También se generará aleatoriamente un entero entre 0 y M-1, que indicará el índice de la máquina a la que se mutará la pieza.

In [142]:
cp = random.randint(0, len(piezas)-1) 
m = random.randint(0, len(tiempos[0])-1)

#se muta la pieza respectiva con el número de la máquina
piezas[cp] = m